In [1]:
import torch
import pytorch_lightning as pl
import torch.nn as nn
import numpy as np
from PIL import Image

In [2]:
# from torchvision.models import resnet50
# from torchvision.models.feature_extraction import get_graph_node_names
# from torchvision.models.feature_extraction import create_feature_extractor

In [3]:
# out

In [4]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import random

# fig, ax = plt.subplots(4, 5, figsize=(25, 20))

# # Pick 4 random feature maps from each layer
# for i, layer in enumerate(feature_model.return_nodes):
#     feat_maps = out[layer].numpy().squeeze(0)
#     feat_maps = random.sample(list(feat_maps), 4)
#     ax[i][0].imshow(input_image)
#     ax[i][0].set_xticks([])
#     ax[i][0].set_yticks([])
#     for j, feat_map in enumerate(feat_maps):
#         sns.heatmap(feat_map, ax=ax[i][j+1], cbar=False)
#         ax[i][j+1].set_xticks([])
#         ax[i][j+1].set_yticks([])
#         ax[i][j+1].set_title(f'{layer}: ({feat_map.shape[0]} x {feat_map.shape[1]})')

In [5]:
""" YELP DATASET """

' YELP DATASET '

In [6]:
import pandas as pd

reviews = pd.read_csv("yelp_dataset/yelp_academic_dataset_review.csv")
businesses = pd.read_csv("yelp_dataset/yelp_academic_dataset_business.csv", converters={'categories': lambda x: set([x.strip() for x in x.split(',')])})
businesses.reset_index(inplace=True)
photos = pd.read_csv("yelp_dataset/photos.csv")
# users = pd.read_csv("yelp_dataset/yelp_academic_dataset_user.csv")

In [7]:
reviews['review_id'] = reviews['review_id'].astype(str)
reviews['user_id'] = reviews['user_id'].astype(str)
reviews['business_id'] = reviews['business_id'].astype(str)
reviews['rating'] = reviews['rating'].astype(int)
reviews['timestamp'] = pd.to_datetime(reviews['timestamp'], format='%Y-%m-%d %H:%M:%S')
reviews.head()

,review_id,user_id,business_id,rating,timestamp
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15


In [8]:
businesses.groupby('city')['review_count'].sum().reset_index().sort_values(by='review_count', ascending=False)

,city,review_count
866,Philadelphia,936240
758,New Orleans,621361
745,Nashville,441053
1157,Tampa,439506
1210,Tucson,387254
...,...,...
1125,Ste C,5
622,MIDDLE CITY WEST,5
1097,St Petersurg,5
1095,St Petersberg,5


In [9]:
businesses = businesses.drop(['state', 'review_count'], axis=1)
businesses

,index,id,name,city,categories
0,0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,"{Traditional Chinese Medicine, Doctors, Nutrit..."
1,1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,"{Mailbox Centers, Local Services, Shipping Cen..."
2,2,tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,"{Shopping, Home & Garden, Furniture Stores, De..."
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,"{Bakeries, Bubble Tea, Food, Restaurants, Coff..."
4,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,"{Brewpubs, Breweries, Food}"
...,...,...,...,...,...
150341,150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,Edmonton,"{Beauty & Spas, Nail Salons}"
150342,150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,Nashville,"{Shopping, Nurseries & Gardening, Pets, Home &..."
150343,150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,Indianapolis,"{Jewelry, Shopping, Toy Stores, Accessories, B..."
150344,150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,Edwardsville,"{Sporting Goods, Shopping, Eyewear & Opticians..."


In [10]:
categories = {"Food", "Coffee & Tea", "Bubble Tea", "Brewpubs", "Sandwiches", "Burgers", "Restaurants", "Fast Food", "Ice Cream & Frozen Yogurt", "Bars", "Pubs", "Sushi Bars"}
# businesses = businesses.drop(['id'], axis=1)
# businesses.columns = ['business_id', 'name', 'city', 'categories']

In [11]:
reviews

,review_id,user_id,business_id,rating,timestamp
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15
...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,2022-01-19 18:59:27


In [12]:
businesses = businesses[businesses.city.isin(('Philadelphia', 'New Orleans', 'Nashville', 'Tampa', 'Tucson'))]
businesses = businesses[businesses['categories'].apply(lambda x: 'Restaurants' in x)]
business_map = {k: v for v, k in enumerate(businesses.index)}
businesses['index'] = businesses['index'].map(business_map)
businesses

,index,id,name,city,categories
3,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,"{Bakeries, Bubble Tea, Food, Restaurants, Coff..."
9,1,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,"{Fast Food, Food, Restaurants, Ice Cream & Fro..."
15,2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,Philadelphia,"{Sushi Bars, Restaurants, Japanese}"
19,3,ROeacJQwBeh05Rqg7F6TCg,BAP,Philadelphia,"{Restaurants, Korean}"
24,4,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,Nashville,"{Restaurants, Hot Dogs}"
...,...,...,...,...,...
150298,16034,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,Philadelphia,"{Restaurants, Italian, Sandwiches}"
150306,16035,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,Philadelphia,"{Sandwiches, Food, Restaurants, Italian, Speci..."
150319,16036,8n93L-ilMAsvwUatarykSg,Kitchen Gia,Philadelphia,"{Sandwiches, Food, Restaurants, American (Trad..."
150322,16037,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,Nashville,"{Fondue, Beer, Food, Restaurants, Wine & Spirits}"


In [13]:
photos = photos.groupby("business_id")["photo_id"].apply(list).reset_index(name="photo_ids")
photos["photo_ids"] = photos["photo_ids"].apply(lambda x: x[0])
photos = pd.merge(photos, businesses, left_on='business_id', right_on='id', how='right')
photos = photos.drop(['id', 'name', 'city', 'categories', 'business_id'], axis = 1)
photos.columns = ['photo_ids', 'business_id']
photos = photos.dropna()
photos

,photo_ids,business_id
0,N3tTxsgpFH91wzo4gZm19A,0
1,ZMd6-8F6W92FWZNz74crUw,1
2,IbkOXfgXq12U5Pye_ySGFw,2
3,oewWFZoEf7AVYyWJ614amw,3
5,W8kiIG3vksNaarHq32TihA,5
...,...,...
16031,3SUzXvOvq646OwnW0zHTxA,16031
16032,s-HhV_HOV7wW4QGfpyptfQ,16032
16035,gilz_mJqz8t0tGr1XCJuXw,16035
16037,zBJS5eNRiov8XgMCs-hsoA,16037


In [14]:
from dataset import Img2Vec

In [15]:
reviews = reviews.drop(['review_id'], axis = 1)
reviews.columns = ['user_id', 'business_id', 'rating', 'timestamp']

In [16]:
reviews

,user_id,business_id,rating,timestamp
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,2015-01-04 00:01:03
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15
...,...,...,...,...
6990275,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,2014-12-17 21:45:20
6990276,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2021-03-31 16:55:10
6990277,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,2019-12-30 03:56:30
6990278,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,2022-01-19 18:59:27


In [17]:
businesses

,index,id,name,city,categories
3,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,"{Bakeries, Bubble Tea, Food, Restaurants, Coff..."
9,1,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,"{Fast Food, Food, Restaurants, Ice Cream & Fro..."
15,2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,Philadelphia,"{Sushi Bars, Restaurants, Japanese}"
19,3,ROeacJQwBeh05Rqg7F6TCg,BAP,Philadelphia,"{Restaurants, Korean}"
24,4,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,Nashville,"{Restaurants, Hot Dogs}"
...,...,...,...,...,...
150298,16034,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,Philadelphia,"{Restaurants, Italian, Sandwiches}"
150306,16035,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,Philadelphia,"{Sandwiches, Food, Restaurants, Italian, Speci..."
150319,16036,8n93L-ilMAsvwUatarykSg,Kitchen Gia,Philadelphia,"{Sandwiches, Food, Restaurants, American (Trad..."
150322,16037,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,Nashville,"{Fondue, Beer, Food, Restaurants, Wine & Spirits}"


In [18]:
photos

,photo_ids,business_id
0,N3tTxsgpFH91wzo4gZm19A,0
1,ZMd6-8F6W92FWZNz74crUw,1
2,IbkOXfgXq12U5Pye_ySGFw,2
3,oewWFZoEf7AVYyWJ614amw,3
5,W8kiIG3vksNaarHq32TihA,5
...,...,...
16031,3SUzXvOvq646OwnW0zHTxA,16031
16032,s-HhV_HOV7wW4QGfpyptfQ,16032
16035,gilz_mJqz8t0tGr1XCJuXw,16035
16037,zBJS5eNRiov8XgMCs-hsoA,16037


In [19]:
df = pd.merge(reviews, businesses, left_on="business_id", right_on="id")
dct = {k: v for v, k in enumerate(df.user_id.unique())}
df.user_id = df.user_id.map(dct)
df

,user_id,business_id,rating,timestamp,index,id,name,city,categories
0,0,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30,1442,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,Tucson,"{Restaurants, Breakfast & Brunch}"
1,1,YjUWPpI6HXG530lwP-fb2A,2,2015-09-10 02:41:13,1442,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,Tucson,"{Restaurants, Breakfast & Brunch}"
2,2,YjUWPpI6HXG530lwP-fb2A,5,2017-08-18 01:42:46,1442,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,Tucson,"{Restaurants, Breakfast & Brunch}"
3,3,YjUWPpI6HXG530lwP-fb2A,5,2015-09-26 16:13:09,1442,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,Tucson,"{Restaurants, Breakfast & Brunch}"
4,4,YjUWPpI6HXG530lwP-fb2A,3,2015-06-22 16:08:27,1442,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,Tucson,"{Restaurants, Breakfast & Brunch}"
...,...,...,...,...,...,...,...,...,...
2042251,498146,x2GJf-ooT8CCtkr-ddUCNA,5,2021-09-29 15:25:10,15363,x2GJf-ooT8CCtkr-ddUCNA,Taqueria La Esquina,Tucson,"{Mexican, Restaurants, Tacos}"
2042252,232763,x2GJf-ooT8CCtkr-ddUCNA,4,2022-01-17 17:47:08,15363,x2GJf-ooT8CCtkr-ddUCNA,Taqueria La Esquina,Tucson,"{Mexican, Restaurants, Tacos}"
2042253,681612,x2GJf-ooT8CCtkr-ddUCNA,1,2021-08-10 17:00:20,15363,x2GJf-ooT8CCtkr-ddUCNA,Taqueria La Esquina,Tucson,"{Mexican, Restaurants, Tacos}"
2042254,297829,x2GJf-ooT8CCtkr-ddUCNA,5,2021-11-09 14:26:54,15363,x2GJf-ooT8CCtkr-ddUCNA,Taqueria La Esquina,Tucson,"{Mexican, Restaurants, Tacos}"


In [20]:
# df = pd.merge(reviews, users[['index', 'user_index']], left_on='user_id', right_on='user_index', how='left')
# df = df.drop(['user_id', 'user_index'], axis=1)
df = df.drop(['business_id', 'name', 'city', 'categories', 'id'], axis=1)
df.columns = ['user_id', 'rating', 'timestamp', 'business_id']
df = df[np.isfinite(df['user_id'])]
dct = {k: v for v, k in enumerate(df.user_id.unique())}
df.user_id = df.user_id.map(dct)
df

,user_id,rating,timestamp,business_id
0,0,3,2014-02-05 20:30:30,1442
1,1,2,2015-09-10 02:41:13,1442
2,2,5,2017-08-18 01:42:46,1442
3,3,5,2015-09-26 16:13:09,1442
4,4,3,2015-06-22 16:08:27,1442
...,...,...,...,...
2042251,498146,5,2021-09-29 15:25:10,15363
2042252,232763,4,2022-01-17 17:47:08,15363
2042253,681612,1,2021-08-10 17:00:20,15363
2042254,297829,5,2021-11-09 14:26:54,15363


In [21]:
# """
# TEST=======================
# """
# # businesses['business_yelp_id'] = businesses['business_yelp_id'].astype(int)
# df = pd.merge(businesses[['index', 'id']], df, left_on="id", right_on="business_id")
# df

In [22]:
# df = df[df['categories'].apply(lambda x: 'Restaurants' in x)]
# df = df.drop(['categories'], axis=1)
# df

In [23]:
# # users = users.reset_index()
# users.drop(['user_name'], axis=1)
# # users.columns = ['user_id', 'user_yelp_id']
# users

In [24]:
# users_temp = df[['index','user_index']]
# users_temp.columns = ['user_id', 'user_yelp_id']
# users_temp = users_temp.drop_duplicates(subset='user_id', keep='first').set_index('user_id')
# users_temp.to_csv("dataset_reset_index/users.csv", index=False, sep=",")
# users_temp

In [25]:
reviews = df.copy()

In [26]:
# df = df.drop(['user_id', 'user_index'], axis=1)
# df.columns = ['review_id', 'rating', 'timestamp', 'business_id', 'user_id']

"""REVIEWS TO CSV"""
# df.to_csv("dataset_reset_index/reviews.csv", index=False, sep=",")
# df

'REVIEWS TO CSV'

In [27]:
photos

,photo_ids,business_id
0,N3tTxsgpFH91wzo4gZm19A,0
1,ZMd6-8F6W92FWZNz74crUw,1
2,IbkOXfgXq12U5Pye_ySGFw,2
3,oewWFZoEf7AVYyWJ614amw,3
5,W8kiIG3vksNaarHq32TihA,5
...,...,...
16031,3SUzXvOvq646OwnW0zHTxA,16031
16032,s-HhV_HOV7wW4QGfpyptfQ,16032
16035,gilz_mJqz8t0tGr1XCJuXw,16035
16037,zBJS5eNRiov8XgMCs-hsoA,16037


In [28]:
df = pd.merge(df, photos, on="business_id", how="left")
df = df.dropna()
df['user_id'] = df['user_id'].astype(int)
df['business_id'] = df['business_id'].astype(int)
# df = df.drop(['business_index'], axis = 1)
# df.columns = ['rating', 'timestamp', 'user_id', 'photo_ids', 'business_id']
df

,user_id,rating,timestamp,business_id,photo_ids
238,228,4,2017-01-14 20:54:15,699,-rCqVHSxxfNSCBLvFE_U6Q
239,229,5,2017-04-30 17:26:37,699,-rCqVHSxxfNSCBLvFE_U6Q
240,230,5,2018-03-07 12:13:26,699,-rCqVHSxxfNSCBLvFE_U6Q
241,231,5,2018-03-07 01:24:05,699,-rCqVHSxxfNSCBLvFE_U6Q
242,232,5,2018-03-16 22:13:07,699,-rCqVHSxxfNSCBLvFE_U6Q
...,...,...,...,...,...
2042251,498146,5,2021-09-29 15:25:10,15363,Oa5r2-yG8KmbAFaeLmaC0A
2042252,232763,4,2022-01-17 17:47:08,15363,Oa5r2-yG8KmbAFaeLmaC0A
2042253,681612,1,2021-08-10 17:00:20,15363,Oa5r2-yG8KmbAFaeLmaC0A
2042254,297829,5,2021-11-09 14:26:54,15363,Oa5r2-yG8KmbAFaeLmaC0A


In [29]:
import time
model = Img2Vec(model="resnet18", cuda=True)

def img_to_vec(photo_id):
    return model.get_vec(Image.open(f'photos/{photo_id}.jpg'))

# photos = photos[~photos.business_id.isin(df.business_id)]
start_time = time.time()
photos['photo_vecs'] = photos.photo_ids.apply(img_to_vec)
print(f'Time to produce {len(photos)} image embeddings: {time.time() - start_time}s')
photos

Time to produce 9604 image embeddings: 118.94838166236877s


C:\Users\Marko\AppData\Local\Temp\ipykernel_15136\2938319564.py:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  photos.photo_vecs = photos.photo_ids.apply(img_to_vec)


,photo_ids,business_id
0,N3tTxsgpFH91wzo4gZm19A,0
1,ZMd6-8F6W92FWZNz74crUw,1
2,IbkOXfgXq12U5Pye_ySGFw,2
3,oewWFZoEf7AVYyWJ614amw,3
5,W8kiIG3vksNaarHq32TihA,5
...,...,...
16031,3SUzXvOvq646OwnW0zHTxA,16031
16032,s-HhV_HOV7wW4QGfpyptfQ,16032
16035,gilz_mJqz8t0tGr1XCJuXw,16035
16037,zBJS5eNRiov8XgMCs-hsoA,16037


In [30]:
reviews_group = df.sort_values(by=["timestamp"]).groupby("user_id")

reviews_data = pd.DataFrame(
    data={
        "user_id": list(reviews_group.groups.keys()),
        "business_ids": list(reviews_group.business_id.apply(list)),
        "ratings": list(reviews_group.rating.apply(list)),
        "timestamps": list(reviews_group.timestamp.apply(list)),
        "photo_ids": list(reviews_group.photo_ids.apply(list)),
    }
)


In [31]:
reviews_data = reviews_data[reviews_data['ratings'].map(len) >= 5]
reviews_data

,user_id,business_ids,ratings,timestamps,photo_ids
0,0,"[1103, 1632, 1632, 7628, 7110, 9970, 1632, 994...","[4, 3, 3, 3, 3, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, ...","[2013-10-13 23:34:57, 2014-02-03 14:34:43, 201...","[id1d2BCegFCGmBxANc-zcA, zNABV85V-SpMGbBBbEK3L..."
2,2,"[15304, 6305, 13069, 210, 15304]","[1, 5, 5, 5, 5]","[2016-09-12 19:15:38, 2017-12-25 22:01:06, 201...","[tBDzjc1WM9-dTq8znx0qvQ, LxDPDd1J8hwcbX8NnQBSD..."
4,4,"[8607, 11518, 5128, 13004, 5013, 9069, 9277, 8...","[5, 5, 3, 4, 3, 4, 3, 4, 4, 3, 5, 4, 3, 4, 3, ...","[2012-05-06 17:56:46, 2012-05-06 18:01:46, 201...","[DYT6tQkLQVs6hTQgh614TQ, 1OLkW2tYShzoI3bD1MVKP..."
12,19,"[2677, 10750, 8096, 579, 13624, 1370, 8079, 82...","[5, 3, 2, 3, 4, 3, 4, 3, 4, 4, 4, 4, 4, 5, 1, ...","[2010-11-08 19:31:13, 2010-11-08 19:45:33, 201...","[JQgbr8DqNaC8xPN6Pk9oHg, Gue0yAQuEiw8gmzS_TT4o..."
13,20,"[8911, 1976, 5235, 11891, 3791, 6878, 1704, 2162]","[5, 5, 5, 5, 5, 5, 5, 1]","[2016-12-11 20:26:04, 2017-01-11 21:06:26, 201...","[vtOtXpGLNqgpcLpRZhnOYg, CxqDfCSC5Zi-2Vd9CrI2H..."
...,...,...,...,...,...
608492,652917,"[13400, 13400, 13400, 13400, 13400]","[5, 4, 4, 3, 3]","[2012-10-30 20:41:33, 2013-07-15 12:56:24, 201...","[bNjeNZEiz6i69ZDvt5qodA, bNjeNZEiz6i69ZDvt5qod..."
617417,663450,"[14879, 14879, 14879, 15357, 14879]","[2, 5, 5, 5, 5]","[2018-08-05 03:50:36, 2018-08-09 00:48:04, 201...","[k6gb_RMvUL1uHoZKAARcLg, k6gb_RMvUL1uHoZKAARcL..."
620956,667050,"[14849, 15364, 15579, 14629, 15579]","[1, 3, 4, 1, 5]","[2016-07-30 16:02:20, 2016-07-30 16:21:03, 201...","[6_XFmgTE5fF5XRCA-ZJ2mg, klB6ILe-wkcddia_yN_1Q..."
636391,683374,"[15843, 15028, 15843, 15843, 15916]","[5, 5, 5, 5, 1]","[2018-09-19 20:22:30, 2020-04-13 00:00:28, 202...","[Rb8X_0mCsBgzvkgqvWwh7A, lzu9MbotKg78Yu8f1iGpa..."


In [32]:
sequence_length = 8
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


reviews_data.business_ids = reviews_data.business_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

reviews_data.ratings = reviews_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

reviews_data.photo_ids = reviews_data.photo_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del reviews_data["timestamps"]

In [33]:
reviews_data = reviews_data[reviews_data['ratings'].map(len) > 0]
reviews_data

,user_id,business_ids,ratings,photo_ids
0,0,"[[1103, 1632, 1632, 7628, 7110, 9970, 1632, 99...","[[4, 3, 3, 3, 3, 4, 3, 5], [3, 3, 3, 4, 3, 5, ...","[[id1d2BCegFCGmBxANc-zcA, zNABV85V-SpMGbBBbEK3..."
4,4,"[[8607, 11518, 5128, 13004, 5013, 9069, 9277, ...","[[5, 5, 3, 4, 3, 4, 3, 4], [3, 4, 3, 4, 3, 4, ...","[[DYT6tQkLQVs6hTQgh614TQ, 1OLkW2tYShzoI3bD1MVK..."
12,19,"[[2677, 10750, 8096, 579, 13624, 1370, 8079, 8...","[[5, 3, 2, 3, 4, 3, 4, 3], [2, 3, 4, 3, 4, 3, ...","[[JQgbr8DqNaC8xPN6Pk9oHg, Gue0yAQuEiw8gmzS_TT4..."
13,20,"[[8911, 1976, 5235, 11891, 3791, 6878, 1704, 2...","[[5, 5, 5, 5, 5, 5, 5, 1], [5, 5, 5, 5, 5, 5, ...","[[vtOtXpGLNqgpcLpRZhnOYg, CxqDfCSC5Zi-2Vd9CrI2..."
15,23,"[[4265, 3312, 8659, 6541, 5667, 3681, 13698, 1...","[[5, 4, 5, 3, 4, 5, 3, 4], [5, 3, 4, 5, 3, 4, ...","[[-p_Z32mVvPEhHsPjpRixMw, 8gdfZLnZ9mdezsvgl6cP..."
...,...,...,...,...
487469,519296,"[[13266, 13852, 10228, 12922, 15339, 15773, 13...","[[3, 4, 4, 4, 3, 5, 3, 5], [3, 4, 4, 4, 3, 5, ...","[[vn1HUh-a0O0nBYXrRdzfzQ, 3x_wFR3zHrTtCznM3ueU..."
491230,523063,"[[15318, 13012, 11191, 12970, 12526, 14196, 12...","[[5, 5, 3, 5, 4, 5, 5, 2], [5, 5, 3, 5, 4, 5, ...","[[g0oadJSwjsoqy6VawbH72A, 70p_Guw_qwiE7U2XeakD..."
491711,523552,"[[14913, 9870, 9870, 9870, 10728, 9870, 9870, ...","[[5, 5, 5, 5, 5, 5, 5, 5], [5, 5, 5, 5, 5, 5, ...","[[3kP0f7-wb4dKS5Ijwag9VQ, 8IYezGv7itKV8aWipu-3..."
494581,526474,"[[12116, 10222, 10222, 13284, 11708, 11132, 12...","[[5, 5, 5, 5, 5, 5, 5, 5], [5, 5, 5, 5, 5, 5, ...","[[56Za2lRcGZpft_2QNKu16Q, NR32fx4bSK4I5_oavfWb..."


In [34]:
# users.columns = ['user_id', 'user_yelp_id', 'user_name']
# users

In [35]:
businesses.columns = ['business_id', 'business_yelp_id', 'name', 'city', 'categories']
businesses

,business_id,business_yelp_id,name,city,categories
3,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,"{Bakeries, Bubble Tea, Food, Restaurants, Coff..."
9,1,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,"{Fast Food, Food, Restaurants, Ice Cream & Fro..."
15,2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,Philadelphia,"{Sushi Bars, Restaurants, Japanese}"
19,3,ROeacJQwBeh05Rqg7F6TCg,BAP,Philadelphia,"{Restaurants, Korean}"
24,4,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,Nashville,"{Restaurants, Hot Dogs}"
...,...,...,...,...,...
150298,16034,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,Philadelphia,"{Restaurants, Italian, Sandwiches}"
150306,16035,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,Philadelphia,"{Sandwiches, Food, Restaurants, Italian, Speci..."
150319,16036,8n93L-ilMAsvwUatarykSg,Kitchen Gia,Philadelphia,"{Sandwiches, Food, Restaurants, American (Trad..."
150322,16037,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,Nashville,"{Fondue, Beer, Food, Restaurants, Wine & Spirits}"


In [36]:
reviews_data_businesses = reviews_data[["user_id", "business_ids"]].explode(
    "business_ids", ignore_index=True
)
reviews_data_photos = reviews_data[["photo_ids"]].explode("photo_ids", ignore_index=True)
reviews_data_rating = reviews_data[["ratings"]].explode("ratings", ignore_index=True)
reviews_data_businesses

,user_id,business_ids
0,0,"[1103, 1632, 1632, 7628, 7110, 9970, 1632, 9947]"
1,0,"[1632, 7628, 7110, 9970, 1632, 9947, 12196, 10..."
2,0,"[7110, 9970, 1632, 9947, 12196, 10174, 3073, 9..."
3,0,"[1632, 9947, 12196, 10174, 3073, 9803, 2616, 1..."
4,0,"[12196, 10174, 3073, 9803, 2616, 11690, 7091, ..."
...,...,...
285663,523552,"[9870, 10728, 9870, 9870, 9870, 9870, 14620, 9..."
285664,526474,"[12116, 10222, 10222, 13284, 11708, 11132, 124..."
285665,526474,"[10222, 10222, 13284, 11708, 11132, 12469, 124..."
285666,532403,"[14069, 14069, 12590, 10635, 13012, 13133, 126..."


In [37]:
# reviews_data_photos = reviews_data[["photo_ids"]].explode("photo_ids", ignore_index=True)
reviews_data_photos

,photo_ids
0,"[id1d2BCegFCGmBxANc-zcA, zNABV85V-SpMGbBBbEK3L..."
1,"[zNABV85V-SpMGbBBbEK3LQ, 7bv-F7pBpRn1DbO00H3YO..."
2,"[q5VyOVnRAPy_wZ5eZwae-w, TCY7WvNwX4dNtldAsIuSy..."
3,"[zNABV85V-SpMGbBBbEK3LQ, ISUJobphjNQqx9X9vRtxu..."
4,"[88S-LwjVS4RAunzLvbN4kw, tsBIYaSorW40KJlpz8wmM..."
...,...
285663,"[8IYezGv7itKV8aWipu-3IA, trUFnQXasV8l2iJSzZKXr..."
285664,"[56Za2lRcGZpft_2QNKu16Q, NR32fx4bSK4I5_oavfWbs..."
285665,"[NR32fx4bSK4I5_oavfWbsw, NR32fx4bSK4I5_oavfWbs..."
285666,"[24L1BQiwDncMeyvAQeaFSg, 24L1BQiwDncMeyvAQeaFS..."


In [38]:
reviews_data_transformed = pd.concat([reviews_data_businesses, reviews_data_rating], axis=1)
reviews_data_transformed = pd.concat([reviews_data_transformed, reviews_data_photos], axis=1)
reviews_data_transformed.business_ids = reviews_data_transformed.business_ids.apply(
    lambda x: ",".join(str(i) for i in x)
)
reviews_data_transformed

,user_id,business_ids,ratings,photo_ids
0,0,"1103,1632,1632,7628,7110,9970,1632,9947","[4, 3, 3, 3, 3, 4, 3, 5]","[id1d2BCegFCGmBxANc-zcA, zNABV85V-SpMGbBBbEK3L..."
1,0,"1632,7628,7110,9970,1632,9947,12196,10174","[3, 3, 3, 4, 3, 5, 5, 5]","[zNABV85V-SpMGbBBbEK3LQ, 7bv-F7pBpRn1DbO00H3YO..."
2,0,"7110,9970,1632,9947,12196,10174,3073,9803","[3, 4, 3, 5, 5, 5, 5, 5]","[q5VyOVnRAPy_wZ5eZwae-w, TCY7WvNwX4dNtldAsIuSy..."
3,0,"1632,9947,12196,10174,3073,9803,2616,11690","[3, 5, 5, 5, 5, 5, 5, 5]","[zNABV85V-SpMGbBBbEK3LQ, ISUJobphjNQqx9X9vRtxu..."
4,0,"12196,10174,3073,9803,2616,11690,7091,9947","[5, 5, 5, 5, 5, 5, 5, 5]","[88S-LwjVS4RAunzLvbN4kw, tsBIYaSorW40KJlpz8wmM..."
...,...,...,...,...
285663,523552,"9870,10728,9870,9870,9870,9870,14620,9870","[5, 5, 5, 5, 5, 5, 1, 5]","[8IYezGv7itKV8aWipu-3IA, trUFnQXasV8l2iJSzZKXr..."
285664,526474,"12116,10222,10222,13284,11708,11132,12469,12469","[5, 5, 5, 5, 5, 5, 5, 5]","[56Za2lRcGZpft_2QNKu16Q, NR32fx4bSK4I5_oavfWbs..."
285665,526474,"10222,10222,13284,11708,11132,12469,12469,10643","[5, 5, 5, 5, 5, 5, 5, 5]","[NR32fx4bSK4I5_oavfWbsw, NR32fx4bSK4I5_oavfWbs..."
285666,532403,"14069,14069,12590,10635,13012,13133,12690,10563","[5, 5, 5, 2, 3, 5, 5, 4]","[24L1BQiwDncMeyvAQeaFSg, 24L1BQiwDncMeyvAQeaFS..."


In [39]:
reviews_data_transformed.columns = ['user_id', 'sequence_business_ids', 'sequence_ratings', 'sequence_photo_ids']
reviews_data_transformed

,user_id,sequence_business_ids,sequence_ratings,sequence_photo_ids
0,0,"1103,1632,1632,7628,7110,9970,1632,9947","[4, 3, 3, 3, 3, 4, 3, 5]","[id1d2BCegFCGmBxANc-zcA, zNABV85V-SpMGbBBbEK3L..."
1,0,"1632,7628,7110,9970,1632,9947,12196,10174","[3, 3, 3, 4, 3, 5, 5, 5]","[zNABV85V-SpMGbBBbEK3LQ, 7bv-F7pBpRn1DbO00H3YO..."
2,0,"7110,9970,1632,9947,12196,10174,3073,9803","[3, 4, 3, 5, 5, 5, 5, 5]","[q5VyOVnRAPy_wZ5eZwae-w, TCY7WvNwX4dNtldAsIuSy..."
3,0,"1632,9947,12196,10174,3073,9803,2616,11690","[3, 5, 5, 5, 5, 5, 5, 5]","[zNABV85V-SpMGbBBbEK3LQ, ISUJobphjNQqx9X9vRtxu..."
4,0,"12196,10174,3073,9803,2616,11690,7091,9947","[5, 5, 5, 5, 5, 5, 5, 5]","[88S-LwjVS4RAunzLvbN4kw, tsBIYaSorW40KJlpz8wmM..."
...,...,...,...,...
285663,523552,"9870,10728,9870,9870,9870,9870,14620,9870","[5, 5, 5, 5, 5, 5, 1, 5]","[8IYezGv7itKV8aWipu-3IA, trUFnQXasV8l2iJSzZKXr..."
285664,526474,"12116,10222,10222,13284,11708,11132,12469,12469","[5, 5, 5, 5, 5, 5, 5, 5]","[56Za2lRcGZpft_2QNKu16Q, NR32fx4bSK4I5_oavfWbs..."
285665,526474,"10222,10222,13284,11708,11132,12469,12469,10643","[5, 5, 5, 5, 5, 5, 5, 5]","[NR32fx4bSK4I5_oavfWbsw, NR32fx4bSK4I5_oavfWbs..."
285666,532403,"14069,14069,12590,10635,13012,13133,12690,10563","[5, 5, 5, 2, 3, 5, 5, 4]","[24L1BQiwDncMeyvAQeaFSg, 24L1BQiwDncMeyvAQeaFS..."


In [40]:
reviews_data_transformed.sequence_photo_ids[0][0]

'id1d2BCegFCGmBxANc-zcA'

In [41]:
import numpy as np
random_selection = np.random.rand(len(reviews_data_transformed.index)) <= 0.8
train_data = reviews_data_transformed[random_selection]
test_data = reviews_data_transformed[~random_selection]

In [42]:
# test_data.to_csv("data/sample_test_data.csv", index=False, sep=",", chunksize=100)

In [43]:
# train_data.to_csv("data/sample_train_data.csv", index=False, sep=",", chunksize=100)

In [44]:
# businesses.to_csv("dataset_reset_index/businesses.csv", index=False, sep=",")
# photos.to_csv("dataset_reset_index/photos.csv", index=False, sep=",")

In [45]:
"""
TEST MODEL
"""

'\nTEST MODEL\n'

In [46]:
import pandas as pd
import torch
import pytorch_lightning as pl
from tqdm import tqdm
import torchmetrics
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from PIL import Image

In [47]:
reviews = pd.read_csv("dataset_reset_index/reviews.csv", sep=",")

In [48]:
from dataset import YelpDataset

In [49]:
sequence_length = 8

In [50]:
class PositionalEmbedding(nn.Module):
    """
    Computes positional embedding following "Attention is all you need"
    """

    def __init__(self, max_len, d_model):
        super().__init__()

        # Compute the positional encodings once in log space.
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size = x.size(0)
        return self.pe.weight.unsqueeze(0).repeat(batch_size, 1, 1)


class BST(pl.LightningModule):
    def __init__(
        self, args=None,
    ):
        super().__init__()
        super(BST, self).__init__()
        
        self.save_hyperparameters()
        self.args = args
        #-------------------
        # Embedding layers
        ##Users      
        self.embeddings_user_id = nn.Embedding(
            int(reviews.user_id.max())+1, int(math.sqrt(reviews.user_id.max()))+1
        )
        
        ##Businesses
        self.embeddings_business_id = nn.Embedding(
            int(businesses.business_id.max())+1, 512
        )
#         self.embeddings_position  = nn.Embedding(
#            sequence_length, int(math.sqrt(len(businesses.business_id.unique())))+1
#         )
        
        self.embeddings_position  = nn.Embedding(
           sequence_length, 512
        )
    
        
#         self.embeddings_photo_id = nn.Embedding(
#             int(photos.photo_ids.max())+1, 512
#         )

        self.positional_embedding = PositionalEmbedding(8, 63)
        
        # Network
        self.transfomerlayer = nn.TransformerEncoderLayer(63, 3, dropout=0.2)
        self.linear = nn.Sequential(
            nn.Linear(
                962,
                32,
            ),
            nn.LeakyReLU(),
            nn.Linear(32, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 1),
        )
        self.criterion = torch.nn.MSELoss()
        self.mae = torchmetrics.MeanAbsoluteError()
        self.mse = torchmetrics.MeanSquaredError()
        


    def encode_input(self,inputs):
        user_id, business_history, photo_history, target_business_id,  business_history_ratings, target_business_rating, target_photo = inputs
#         user_id, business_history, target_business_id,  business_history_ratings, target_business_rating = inputs
                       
        #MOVIES
        business_history = self.embeddings_business_id(business_history)
        target_business = self.embeddings_business_id(target_business_id)
#         photo_history = torch.unsqueeze(photo_history, 1)
        
        positions = torch.arange(0,sequence_length-1,1,dtype=int,device=self.device)
        positions = self.embeddings_position(positions)
        
        target_photo = torch.unsqueeze(target_photo, 1)
        target_business = torch.unsqueeze(target_business, 1)
        
        encoded_sequence_businesses_with_position_and_rating = (business_history + positions + photo_history) * business_history_ratings[..., None]
        
#         print(f'Features shape {encoded_sequence_businesses_with_position_and_rating.shape}')
#         print(f'Business shape {target_business.shape}')
    
        transformer_features_x = torch.cat((encoded_sequence_businesses_with_position_and_rating, target_business), 1)
        transfomer_features = torch.cat((transformer_features_x, target_photo), 1)
#         transfomer_features = torch.cat((encoded_sequence_businesses_with_position_and_rating, target_business), 1)
        
        #USERS
        user_id = self.embeddings_user_id(user_id)
        
        user_features = user_id
        
        return transfomer_features, user_features, target_business_rating.float()
    
    def forward(self, batch):
        transfomer_features, user_features, target_business_rating = self.encode_input(batch)
        transfomer_features = self.positional_embedding(transfomer_features)
        transformer_output = self.transfomerlayer(transfomer_features)
        transformer_output = torch.flatten(transformer_output,start_dim=1)
        
        #Concat with other features
        features = torch.cat((transformer_output,user_features),dim=1)

        output = self.linear(features)
        return output, target_business_rating
        
    def training_step(self, batch, batch_idx):
        out, target_business_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_business_rating)
        
        mae = self.mae(out, target_business_rating)
        mse = self.mse(out, target_business_rating)
        rmse =torch.sqrt(mse)
        self.log(
            "train/mae", mae, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log(
            "train/rmse", rmse, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log("train/step_loss", loss, on_step=True, on_epoch=False, prog_bar=False)
        return loss
    
    def validation_step(self, batch, batch_idx):
        out, target_business_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_business_rating)
        
        mae = self.mae(out, target_business_rating)
        mse = self.mse(out, target_business_rating)
        rmse =torch.sqrt(mse)
        
        return {"val_loss": loss, "mae": mae.detach(), "rmse":rmse.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_mae = torch.stack([x["mae"] for x in outputs]).mean()
        avg_rmse = torch.stack([x["rmse"] for x in outputs]).mean()
        
        self.log("val/loss", avg_loss, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/mae", avg_mae, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/rmse", avg_rmse, on_step=False, on_epoch=True, prog_bar=False)


    def test_epoch_end(self, outputs):
        users = torch.cat([x["users"] for x in outputs])
        y_hat = torch.cat([x["top14"] for x in outputs])
        users = users.tolist()
        y_hat = y_hat.tolist()
        
        data = {"users": users, "top14": y_hat}
        df = pd.DataFrame.from_dict(data)
        df.to_csv("lightning_logs/predict.csv", index=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0005)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument("--learning_rate", type=float, default=0.01)
        return parser

    ####################
    # DATA RELATED HOOKS
    ####################

    def setup(self, stage=None):
        print("Loading datasets")
#         self.train_dataset = YelpDataset("data/sample_train_data.csv")
#         self.val_dataset = YelpDataset("data/sample_test_data.csv")
#         self.test_dataset = YelpDataset("data/sample_test_data.csv")
        
        self.train_dataset = YelpDataset(train_data)
        self.val_dataset = YelpDataset(test_data)
        self.test_dataset = YelpDataset(test_data)
        print("Done")

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=0
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=0
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=0
        )

In [51]:
# trainer = pl.Trainer(accelerator='cpu', max_epochs=10)
# trainer.fit(model)

In [52]:
model_gpu = BST()
print(model_gpu)

ValueError: invalid literal for int() with base 10: 'zzzMBVS73g3ZJ7qL8JyhiA'

In [ ]:
%load_ext tensorboard

In [ ]:
trainer_gpu = pl.Trainer(accelerator='gpu', devices=1, max_epochs=30)
trainer_gpu.fit(model_gpu)

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter("runs/gpu_30")